In [46]:
import unicodedata
from pathlib import Path
!pip install scikit-learn==1.3.0

import os

!pip install catboost
import catboost as cb
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
!pip install category_encoders
from category_encoders import CountEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import TargetEncoder

from geopy.geocoders import Nominatim
# geolocator = Nominatim()

from geopy.extra.rate_limiter import RateLimiter
import time


import numpy as np
import pandas as pd
!pip install catboost
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
import os

import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error



In [47]:
# MAPEの計算関数
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_indices = y_true != 0
    return np.mean(np.abs((y_true[non_zero_indices] - y_pred[non_zero_indices]) / y_true[non_zero_indices])) * 100

In [48]:
INPUT_DIR = '/content/drive/MyDrive/signate/Student Cup 2023/input'

# データの読み込み
data = pd.read_csv(os.path.join(INPUT_DIR, 'train.csv'))

test_df = pd.read_csv(os.path.join(INPUT_DIR, "test.csv"))
sub_df = pd.read_csv(os.path.join(INPUT_DIR, "submit_sample.csv"), names=["id", "price"])


In [49]:
# region -> stateが一意に定まることを確認
region_state = {region:{} for region in data['region'].unique()}
for row, value in data.iterrows():
    if not pd.isna(value['state']):
        if value['state'] not in region_state[value['region']]:
            region_state[value['region']][value['state']] = 1
        else:
            region_state[value['region']][value['state']] += 1

for region, state_dict in region_state.items():
    if len(state_dict) > 1 or state_dict == {}:
        region_state[region] = pd.NA
    else:
        region_state[region] = list(state_dict.keys())[0]

# regionからstateを決定
data['state'] = [region_state[region] if pd.isna(state) else state for region, state in zip(data['region'], data['state'])]
test_df['state'] = [region_state[region] if pd.isna(state) else state for region, state in zip(test_df['region'], test_df['state'])]


In [50]:
agg_col = ["condition", "fuel", "transmission"]
# manufacturer  paint_color v

for x in agg_col:
    grouped = data.groupby(x)['odometer']
    agg_data = grouped.agg(
        **{
            f"{x}_median_odometer": 'median',
            f"{x}_mean_odometer": 'mean'
        }
    ).reset_index()

    # 元のデータフレームに計算結果をマージ
    data = data.merge(agg_data, on=x, how='left')
    test_df = test_df.merge(agg_data, on=x, how='left')

In [57]:
class PreProcessTransformer(TransformerMixin, BaseEstimator):
  def fit(self, X, y=None):
    return self

  def transform(self, X):
    # X["title_status"] = X["title_status"].fillna("clean")

    # cylinderから数値を取り出す
    X["cylinders"] = X["cylinders"].astype(str).str.extract("(\d+)") #.astype("float32")
    # X["cylinders"] = X["cylinders"].astype(str).str.extract("(\d+)").fillna(0).astype("float32")


    # sizeの表記揺れを修正
    X["size"] = X["size"].str.replace("ー", "-").astype(str)
    X["size"] = X["size"].str.replace("−", "-").astype(str)

    # 順位付け: サイズ別に値を割り当てます。大きいほど高い値にします。
    ranking_dict = {'full-size': 4, 'mid-size': 3, 'compact': 2, 'sub-compact': 1}

    # 新しい順位カラムを作成します。
    X['size_rank'] = X['size'].map(ranking_dict)

    # manufacturerの表記揺れを修正
    X["manufacturer"] = X["manufacturer"].apply(
        lambda x: unicodedata.normalize("NFKC", x).lower()
    )

    # 正規化の辞書
    normalize_dict = {
        'niѕsan': 'nissan',
        'subαru': 'subaru',
        'toyotа': 'toyota',
        'sαturn': 'saturn',
        'аcura': 'acura',
        'vоlkswagen': 'volkswagen',
        'lexuѕ': 'lexus',
        'ᴄhrysler': 'chrysler'
    }


    # データフレームのメーカー名を正規化
    X['manufacturer'] = X['manufacturer'].map(lambda x: normalize_dict.get(x, x))

    # 国別分類
    def classify_country(manufacturer):
        if pd.isnull(manufacturer):
            return 'Unknown'
        if manufacturer in ['bmw', 'volkswagen', 'audi', 'mercedes-benz', 'porsche']:
            return 'Germany'
        elif manufacturer in ['toyota', 'nissan', 'lexus', 'subaru', 'honda', 'mazda', 'mitsubishi', 'infiniti', 'acura']:
            return 'Japan'
        elif manufacturer in ['ford', 'buick', 'ram', 'dodge', 'chevrolet', 'jeep', 'chrysler', 'gmc', 'lincoln', 'cadillac', 'pontiac', 'saturn']:
            return 'USA'
        elif manufacturer in ['hyundai', 'kia']:
            return 'South Korea'
        elif manufacturer in ['fiat', 'alfa-romeo']:
            return 'Italy'
        elif manufacturer in ['land rover', 'jaguar', 'aston-martin', 'rover']:
            return 'UK'
        else:
            return 'Unknown'

    X['country'] = X['manufacturer'].apply(classify_country)


    def classify_color(color):
        if color in ['orange', 'yellow', 'red', 'white']:
            return 'Bright'
        elif color in ['black', 'brown', 'green', 'blue']:
            return 'Dark'
        elif color in ['silver', 'grey']:
            return 'Neutral'
        elif color in ['purple', 'custom']:
            return 'Special'
        else:
            return 'Unknown'

    X['color_category'] = X['paint_color'].apply(classify_color)

    # one-hot encoding
    # X = pd.get_dummies(X, columns=['country'])

    # yearが3000年以降のものはおかしいので2000年に置換
    err_idx = X.query("year >= 3000").index
    X.loc[err_idx, "year"] = X.loc[err_idx, "year"].apply(lambda x: x - 1000)

    # 走行距離 / 製造年
    X["odometer_per_year"] = X["odometer"] / (2023 - X["year"])

    for col in agg_col:
      X[f"{col}_diff_mean"] = X["odometer"] - X[f"{x}_mean_odometer"]
      X[f"{col}_diff_median"] = X["odometer"] - X[f"{x}_median_odometer"]

    # X["pass_year"] = 2023 - X["year"]

    # X['state'] = X['state'].replace('<NA>', 'null')
    # X.dropna(subset=['state'], inplace=True)
    X['state'] = X['state'].fillna("unknown")

    err_idx = X.query("odometer < -1").index
    X.loc[err_idx, "odometer"] = X.loc[err_idx, "odometer"].apply(lambda x: -1)

    # # Geolocatorオブジェクトを作成
    # geolocator = Nominatim(user_agent="gabikueh")

    # # ユニークなregionを取得
    # unique_regions = X['region'].unique()

    # # 緯度経度の辞書を作成
    # lat_lon_dict = {}

    # for city in unique_regions:
    #     try:
    #         location = geolocator.geocode(city)
    #         if location:
    #             lat_lon_dict[city] = {'latitude': location.latitude, 'longitude': location.longitude}
    #             print(location.raw)  # ジオコーディングした結果の全ての情報
    #         else:
    #             lat_lon_dict[city] = {'latitude': None, 'longitude': None}
    #             print(f"Failed to geocode for city: {city}")
    #     except Exception as e:
    #         lat_lon_dict[city] = {'latitude': None, 'longitude': None}
    #         print(f"Error geocoding city: {city}. Error message: {e}")
    #     time.sleep(2)  # 1秒待機

    # # ユニークな地域の緯度経度を元のデータフレームにマージ
    # geo_df = pd.DataFrame.from_dict(lat_lon_dict, orient='index').reset_index().rename(columns={'index': 'region'})
    # X = X.merge(geo_df, on='region', how='left')

    return X

class OriginalTransformer(TransformerMixin, BaseEstimator):
    """数値特徴はそのまま、カテゴリ特徴はcategory型に変換"""

    def __init__(self, numeric_cols, categorical_cols):
        self.numeric_cols = numeric_cols
        self.categorical_cols = categorical_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_new = X.copy()

        # 数値変数
        X_new[self.numeric_cols] = X_new[self.numeric_cols].astype("float32")

        # カテゴリ変数
        X_new[self.categorical_cols] = X_new[self.categorical_cols].astype("category")

        return X_new[self.get_feature_names_out()]

    def get_feature_names_out(self):
        return self.numeric_cols + self.categorical_cols

In [58]:
cat_features = [ 'ori__region',
                'ori__cylinders',
                'ori__manufacturer',
                 'ori__condition',
                 'ori__fuel',
                 'ori__title_status',
                 'ori__transmission',
                 'ori__drive',
                 'ori__size',
                 'ori__type',
                 'ori__paint_color',
                 'ori__state',
                 'ori__country',
                 'ori__color_category'
                 ]


In [59]:
result_agg_cols = [f"{x}_diff_{stat}" for x in agg_col for stat in ['mean', 'median']]
numeric_cols = ["year", "odometer", "odometer_per_year", "size_rank"] + result_agg_cols
categorical_cols = [
    'region',
    "cylinders",
    "manufacturer",
    "condition",
    "fuel",
    "title_status",
    "transmission",
    "drive",
    "size",
    "type",
    "paint_color",
    "state",
    "country",
    "color_category"
]

ct = ColumnTransformer(
    transformers=[
        (
            "ori",
            OriginalTransformer(numeric_cols, categorical_cols),
            categorical_cols + numeric_cols,
        ),
        (
            "tgt",
            TargetEncoder(target_type="continuous", random_state=88),
            [
                'region',
                "cylinders",
                "manufacturer",
                "condition",
                "fuel",
                "title_status",
                "transmission",
                "drive",
                "size",
                "type",
                "paint_color",
                "state",
                "country",
                "color_category"
            ],
        ),
        # *[
        #     (f"agg_{key}", AggTransformer(key, ["odometer"], {"mean"}), [key] + ["odometer"])
        #     for key in [
        #         "manufacturer",
        #     ]
        # ],
    ],
    verbose=True,
)
ct.set_output(transform="pandas")
pipe = Pipeline(
    steps=[
        ("preprocess", PreProcessTransformer()),
        ("ct", ct),
    ]
)


In [65]:
from sklearn.model_selection import KFold
import numpy as np

# target変数と説明変数に分割
X = pipe.fit_transform(data, data["price"])
test_feat_df = pipe.transform(test_df)
X = X.dropna()

# Xで削除した行をyからも削除
y = data.loc[X.index, 'price']

# # カテゴリ変数を文字列に変換
# for feature in cat_features:
#     X[feature] = X[feature].astype(str)

# 5-fold クロスバリデーションの準備
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mape_scores = []

# パラメータチューニング
params = {
    'loss_function': 'MAPE',
    'eval_metric': 'MAPE',
    'verbose': 200,
    'depth': 7,          # 木の深さ
    'learning_rate': 0.05, # 学習率
    'iterations': 1500,    # 反復回数
    'l2_leaf_reg': 3.0,    # L2正則化
    'border_count': 128,   # ビニングの数
    'random_strength': 0.8, # ランダム性の度合い
    'bagging_temperature': 0.5 # バギングの温度
}

# 各フォールドに対してモデルのトレーニングと評価を行う
for train_index, valid_index in kf.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    train_pool = Pool(X_train, y_train, cat_features=cat_features)
    valid_pool = Pool(X_valid, y_valid, cat_features=cat_features)

    model = CatBoostRegressor(**params)
    model.fit(train_pool, eval_set=valid_pool)

    y_pred = model.predict(valid_pool)
    mape = mean_absolute_error(y_valid, y_pred) / y_valid.mean() * 100
    mape_scores.append(mape)

# 全フォールドのMAPEの平均と標準偏差を表示
print(f"Average Validation MAPE: {np.mean(mape_scores):.2f}%")
print(f"Standard Deviation of Validation MAPE: {np.std(mape_scores):.2f}%")

[ColumnTransformer] ........... (1 of 2) Processing ori, total=   0.1s
[ColumnTransformer] ........... (2 of 2) Processing tgt, total=   0.1s
0:	learn: 0.5665249	test: 0.5592614	best: 0.5592614 (0)	total: 85.3ms	remaining: 2m 7s
200:	learn: 0.5499311	test: 0.5484471	best: 0.5484471 (200)	total: 16.4s	remaining: 1m 45s
400:	learn: 0.5460672	test: 0.5471933	best: 0.5471645 (391)	total: 31.5s	remaining: 1m 26s
600:	learn: 0.5432811	test: 0.5458095	best: 0.5458091 (599)	total: 46.2s	remaining: 1m 9s
800:	learn: 0.5413610	test: 0.5452676	best: 0.5452511 (795)	total: 1m	remaining: 53.1s
1000:	learn: 0.5399515	test: 0.5454012	best: 0.5452511 (795)	total: 1m 18s	remaining: 38.9s
1200:	learn: 0.5389046	test: 0.5452514	best: 0.5451671 (1099)	total: 1m 33s	remaining: 23.3s
1400:	learn: 0.5377190	test: 0.5452557	best: 0.5451671 (1099)	total: 1m 49s	remaining: 7.73s
1499:	learn: 0.5372880	test: 0.5454592	best: 0.5451671 (1099)	total: 1m 56s	remaining: 0us

bestTest = 0.5451670846
bestIteration = 10

In [82]:
# 数値データのカラムの欠損値を平均で補完
for col in test_feat_df.select_dtypes(include=['float', "int"]).columns:
    print(col)
    test_feat_df[col].fillna(0, inplace=True)
print("_________")
# カテゴリデータのカラムの欠損値を"unknown"で補完
for feature in cat_features:
    print(feature)
    test_feat_df[feature] = test_feat_df[feature].astype(str).fillna("unknown")


# inference
pred = np.array(model.predict(test_feat_df))

# submission
sub_df["price"] = pred
sub_df.to_csv("submission-026.csv", index=False, header=False)

ori__year
ori__odometer
ori__odometer_per_year
ori__size_rank
ori__condition_diff_mean
ori__condition_diff_median
ori__fuel_diff_mean
ori__fuel_diff_median
ori__transmission_diff_mean
ori__transmission_diff_median
tgt__region
tgt__cylinders
tgt__manufacturer
tgt__condition
tgt__fuel
tgt__title_status
tgt__transmission
tgt__drive
tgt__size
tgt__type
tgt__paint_color
tgt__state
tgt__country
tgt__color_category
_________
ori__region
ori__cylinders
ori__manufacturer
ori__condition
ori__fuel
ori__title_status
ori__transmission
ori__drive
ori__size
ori__type
ori__paint_color
ori__state
ori__country
ori__color_category


In [83]:
sub_df

,id,price
0,27532,5746.392722
1,27533,5401.662069
2,27534,5451.038468
3,27535,6130.077462
4,27536,4735.691866
...,...,...
27532,55064,5556.235152
27533,55065,5333.463921
27534,55066,5359.515708
27535,55067,5436.532517
